In [1]:
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import requests
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv("C:\parser\gutenberg_metadata.csv")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\User\AppData\Local\Temp\ipykernel_17608\4226708652.py:1: SyntaxWarning: invalid escape sequence '\p'
  data = pd.read_csv("C:\parser\gutenberg_metadata.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15331 entries, 0 to 15330
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      15331 non-null  object
 1   Author     14135 non-null  object
 2   Link       15331 non-null  object
 3   Bookshelf  2732 non-null   object
dtypes: object(4)
memory usage: 479.2+ KB


In [4]:
data.Bookshelf.value_counts().head(15)

Bookshelf
Children's          213
FR                   84
The                  77
United               61
Science              50
World                50
Crime                50
IT                   33
Architecture         25
Gothic               25
Art                  25
Reference            25
Bahá'í               25
Animals-Domestic     25
Folklore             25
Name: count, dtype: int64

В данных много текстов нехудожественной литературы. Ниже приведен список основных жанров, которые нужно исключить.

In [10]:
genres_exclude = ['Animal', 'Animals-Domestic', 'Animals-Wild','Animals-Wild-Trapping', 'Animals-Wild-Birds', 'Animals-Wild-Insects',
                  'Animals-Wild-Mammals', 'Animals-Wild-Reptiles', 'Science', 'Architecture', 'Art', 'IT', 'Anthropology', 'Archaeology', 'Psychology', 'Philosophy',
                 'Ecology', 'Zoology', 'Astronomy', 'Cookbooks', 'Technology', 'Botany', 'Physics', 'Anarchism', 'Crafts', 'Scientific', 'Biology', 'Transportation',
                 'Woodwork', 'Mathematics', 'Sociology', 'Manufacturing', 'Chemistry', 'Slavery', 'Engineering', 'Travel', 'Language', 'CIA', 'Camping',
                 'Maps', 'Music', 'Buddhism', 'Judaism', 'Islam', 'Hinduism', 'Mycology', 'Reference', 'Christianity', 'Noteworthy',  'Atheism',
                  'Racism', 'Microscopy', 'Geology', 'Education', 'Paganism', 'Bahá\'í', 'Journal', 'Current', 'Witchcraft', 'Biographies', 'General',
                  'Latter', 'Canon', 'Microbiology', 'Suffrage', 'Boer', 'Biographies', 'Physiology',
                 ]

In [12]:
# Художественные жанры
data_fiction = data.query('~Bookshelf.isin(@genres_exclude)')

Ссылка на html страницу с текстом книги имеет следующую форму: https://www.gutenberg.org/cache/epub/27730/pg27730
В неё `27730` - это номер книги. Создаем отдельный принзак со ссылкой на текст книги.

In [15]:
data_fiction['book_number'] = data_fiction['Link'].str.split('/').apply(lambda x: x[-1])

link = 'https://www.gutenberg.org/cache/epub/'
end_link = '/pg'

data_fiction['text_link'] = link + data_fiction['book_number'] + end_link + data_fiction['book_number']
data_fiction.head(5)

C:\Users\User\AppData\Local\Temp\ipykernel_17608\2209793601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fiction['book_number'] = data_fiction['Link'].str.split('/').apply(lambda x: x[-1])
C:\Users\User\AppData\Local\Temp\ipykernel_17608\2209793601.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fiction['text_link'] = link + data_fiction['book_number'] + end_link + data_fiction['book_number']


,Title,Author,Link,Bookshelf,book_number,text_link
200,Anne of Green Gables,L. M. Montgomery,http://www.gutenberg.org/ebooks/45,Canada,45,https://www.gutenberg.org/cache/epub/45/pg45
201,Anne of Avonlea,L. M. Montgomery,http://www.gutenberg.org/ebooks/47,Canada,47,https://www.gutenberg.org/cache/epub/47/pg47
202,Anne of the Island,L. M. Montgomery,http://www.gutenberg.org/ebooks/51,Canada,51,https://www.gutenberg.org/cache/epub/51/pg51
203,Anne's House of Dreams,L. M. Montgomery,http://www.gutenberg.org/ebooks/544,Canada,544,https://www.gutenberg.org/cache/epub/544/pg544
204,Rilla of Ingleside,L. M. Montgomery,http://www.gutenberg.org/ebooks/3796,Canada,3796,https://www.gutenberg.org/cache/epub/3796/pg3796


Удалим явные дубликаты.

In [18]:
print('Явных дубликатов - ', data_fiction.duplicated().sum())
data_fiction = data_fiction.drop_duplicates()
print('Явных дубликатов после удаления - ', data_fiction.duplicated().sum())

Явных дубликатов -  5565
Явных дубликатов после удаления -  0


Итерируемся по ссылкам и извлекаем тексты книг в отдельный dataframe.

In [29]:
agent = UserAgent()
headers = {'UserAgent': agent.random}
text_info = []
counter = 1
for title, author, url in tqdm(zip(data_fiction['Title'], data_fiction['Author'], data_fiction['text_link'])): 
    # Получаем информацию от сервера
    try:
        with requests.get(url, headers=headers) as response:
            response = response
    except Exception as e:
        print(f'Возникла ошибка {e}')


    # Извлекаем текст книги
    text = response.text
    # Удаляем лишние символы
    ##### text = text.replace('\r', '').replace('\n', '').replace('*', '').replace('_', '')

    # Заполняем словарь и добавляем его в список
    text_dict = {}
    text_dict['title'] = title
    text_dict['author'] = author
    text_dict['text'] = text
    text_info.append(text_dict)


    # Сохраняем каждые 100 книг на всякий случай
    if counter % 100 == 0:
        pd.DataFrame(text_info).to_csv(f'checkpoint{counter//100}.csv')
        print('===============================')
        print(f'Checkpoint{counter//100} is done')
        print('===============================')

    if counter % 50 == 0: 
        print(f'{counter} итераций сделано')

    counter += 1

    # Спим случайное время
    sleep_time = np.random.choice([0.5, 0.6, 0.7, 0.8, 0.9]) 
    time.sleep(sleep_time)


books_text_final = pd.DataFrame(text_info)
books_text_final.to_csv('books_text_final_new.csv') 

0it [00:00, ?it/s]

50 итераций сделано
Checkpoint1 is done
100 итераций сделано
150 итераций сделано
Checkpoint2 is done
200 итераций сделано
250 итераций сделано
Checkpoint3 is done
300 итераций сделано
350 итераций сделано
Checkpoint4 is done
400 итераций сделано
450 итераций сделано
Checkpoint5 is done
500 итераций сделано
550 итераций сделано
Checkpoint6 is done
600 итераций сделано
650 итераций сделано
Checkpoint7 is done
700 итераций сделано
750 итераций сделано
Checkpoint8 is done
800 итераций сделано
850 итераций сделано
Checkpoint9 is done
900 итераций сделано
950 итераций сделано
Checkpoint10 is done
1000 итераций сделано
1050 итераций сделано
Checkpoint11 is done
1100 итераций сделано
1150 итераций сделано
Checkpoint12 is done
1200 итераций сделано
1250 итераций сделано
Checkpoint13 is done
1300 итераций сделано
1350 итераций сделано
Checkpoint14 is done
1400 итераций сделано
1450 итераций сделано
Checkpoint15 is done
1500 итераций сделано
1550 итераций сделано
Checkpoint16 is done
1600 итерац

Формируем датафрейм из названий книг, авторов и текстов.

In [32]:
text_info = pd.read_csv('books_text_final_new.csv')
text_info = text_info.drop('Unnamed: 0', axis=1)
text_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8466 entries, 0 to 8465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   8466 non-null   object
 1   author  7704 non-null   object
 2   text    8466 non-null   object
dtypes: object(3)
memory usage: 198.6+ KB


In [33]:
text_info.head()

,title,author,text
0,Anne of Green Gables,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of Green ...
1,Anne of Avonlea,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of Avonle...
2,Anne of the Island,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of the Is...
3,Anne's House of Dreams,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne's House o...
4,Rilla of Ingleside,L. M. Montgomery,﻿The Project Gutenberg eBook of Rilla of Ingle...


In [30]:
text_info = text_info.drop('Unnamed: 0', axis=1)

,title,author,text
0,Anne of Green Gables,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of Green ...
1,Anne of Avonlea,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of Avonle...
2,Anne of the Island,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne of the Is...
3,Anne's House of Dreams,L. M. Montgomery,﻿The Project Gutenberg eBook of Anne's House o...
4,Rilla of Ingleside,L. M. Montgomery,﻿The Project Gutenberg eBook of Rilla of Ingle...
...,...,...,...
8461,Psichopath,Randall Garrett,﻿The Project Gutenberg eBook of Psichopath\r\n...
8462,Poisoned Air,S. P. Meek,﻿The Project Gutenberg eBook of Poisoned Air\r...
8463,Cry from a Far Planet,Tom Godwin,﻿The Project Gutenberg eBook of Cry from a Far...
8464,Spies Die Hard!,Arnold Marmor,﻿The Project Gutenberg eBook of Spies Die Hard...
